In [1]:
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from rasterio.mask import mask
from rasterio import features
import warnings

warnings.filterwarnings("ignore")
print("BE CAREFULL warning are masked !!")

BE CAREFULL warning are masked !!


import dataset

In [2]:
EnMAP_glob_path = "images/EnMAP_resampled_glob.tif"

S2_glob_path = "images/Stack_im_resampled_glob.tif"

glob_mask_path = "images/out/4.tif"

shp_path = "images/DECL_23_crop.shp"

Check crs

In [3]:
gdf = gpd.read_file(shp_path)
src = rasterio.open(EnMAP_glob_path, "r")

crs_vector = str(gdf.crs).split(":", 1)[1]
crs_raster = str(src.crs).split(":", 1)[1]

if crs_vector == crs_raster:
    print(f"CRS are the same : EPSG:{crs_vector} = EPSG:{crs_raster}")
else:
    print("We must reproject vector file")
    gdf = gdf.to_crs(epsg=crs_raster)
    print("vector file has been well reprojected")

CRS are the same : EPSG:32631 = EPSG:32631


Applying buffer

In [4]:
gdf.geometry = gdf.geometry.buffer(-30)

print(f"The Coordinates Reference System is {gdf.crs} \n")

print(
    f"There are {len(gdf)} polygons in the EnMAP area BEFORE applying the {-30}m buffer."
)

gdf.geometry = gdf.geometry.buffer(-30)

gdf = gdf[~gdf.geometry.is_empty]  # Remove empty geometries

print(f"There are {len(gdf)} polygons AFTER applying the {30}m buffer.")

The Coordinates Reference System is EPSG:32631 

There are 20256 polygons in the EnMAP area BEFORE applying the -30m buffer.
There are 6167 polygons AFTER applying the 30m buffer.


filter gdf dataset

In [5]:
code_list = ["311", "321", "331", "341", "351", "36"]

gdf_filtered = gdf.loc[gdf["CULT_COD"].isin(code_list)]
gdf_filtered["area"] = gdf_filtered.geometry.area.astype(int)

gdf_filtered = gdf_filtered.reset_index()

Compute real area of each polygon, taking into account no data values

In [6]:
### Doing it with S2 img is the same than for EnMAP img.

src = rasterio.open(S2_glob_path, "r")
profile = src.profile
profile.update(dtype=rasterio.float32, count=1, compress="lzw")
im_arr = src.read(3)

src_cloud = rasterio.open(glob_mask_path)

CLOUD_MASK = src_cloud.read(1)

im_masked = CLOUD_MASK * im_arr

columns = ["index"]
stats = ["count", "mean", "std", "nodata"]

# Get transform from profile (metadata)
profile = src.profile
transform = profile["transform"]

src.close()

df_masked = pd.DataFrame(
    zonal_stats(
        gdf_filtered,
        im_masked,
        affine=transform,
        stats=["count", "mean", "std", "nodata"],
        nodata=0,
        geojson_out=True,
    )
)

df_masked = pd.json_normalize(df_masked["properties"])[columns + stats]
df_masked = df_masked.loc[df_masked["count"] > 0]

src.close()
src_cloud.close()

In [7]:
gdf_masked = pd.merge(gdf_filtered, df_masked, on="index", how="inner")
gdf_masked["real_area"] = gdf_masked["area"] * (
    gdf_masked["count"] / (gdf_masked["count"] + gdf_masked["nodata"])
)

Split 80/20

In [8]:
# Shuffle the GeoDataFrame rows
gdf_masked = gdf_masked.sample(frac=1, random_state=42).reset_index(drop=True)


# Function to perform the split for a specific class
def split_class(gdf, desired_area_fraction=0.8):
    # Calculate the total area for the class
    total_area_class = gdf["real_area"].sum()

    # Desired area for the 80% subset
    desired_area = desired_area_fraction * total_area_class

    # Initialize variables to track the cumulative area and the index to split
    cumulative_area = 0
    split_index = 0

    # Iterate over the rows to find the split point
    for i, area in enumerate(gdf["real_area"]):
        cumulative_area += area
        if cumulative_area >= desired_area:
            split_index = i + 1
            break

    # Split the GeoDataFrame
    gdf_80 = gdf.iloc[:split_index]
    gdf_20 = gdf.iloc[split_index:]

    # Ensure there's at least 1 polygon in the 20% subset if the class has more than 1 polygon
    if len(gdf_20) == 0 and len(gdf) > 1:
        gdf_80 = gdf.iloc[:-1]
        gdf_20 = gdf.iloc[[-1]]  # Move the last polygon from the 80% to 20%

    return gdf_80, gdf_20


# List to store the subsets
gdf_80_list = []
gdf_20_list = []

# Loop over each class
for cult_cod in gdf_masked["CULT_COD"].unique():
    # Filter the GeoDataFrame for the current class
    gdf_class = gdf_masked[gdf_masked["CULT_COD"] == cult_cod]

    # Split the class-specific GeoDataFrame
    gdf_80_class, gdf_20_class = split_class(gdf_class)

    # Append the results to the lists
    gdf_80_list.append(gdf_80_class)
    gdf_20_list.append(gdf_20_class)

# Concatenate all the class-specific subsets to form the final GeoDataFrames
gdf_80 = pd.concat(gdf_80_list).reset_index(drop=True)
gdf_20 = pd.concat(gdf_20_list).reset_index(drop=True)

# Print the results
total_area = gdf_masked["real_area"].sum()
desired_area = 0.8 * total_area

print(f"Total area: {total_area}")
print(f"Desired 80% area: {desired_area}")
print(f'Actual 80% subset area: {gdf_80["real_area"].sum()}')
print(f"Number of polygons in 80% subset: {len(gdf_80)}")
print(f"Number of polygons in 20% subset: {len(gdf_20)}")

# Verify the split for each class
for cult_cod in gdf_masked["CULT_COD"].unique():
    class_total_area = gdf_masked[gdf_masked["CULT_COD"] == cult_cod][
        "real_area"
    ].sum()
    class_80_area = gdf_80[gdf_80["CULT_COD"] == cult_cod]["real_area"].sum()
    class_20_area = gdf_20[gdf_20["CULT_COD"] == cult_cod]["real_area"].sum()
    class_80_polygons = len(gdf_80[gdf_80["CULT_COD"] == cult_cod])
    class_20_polygons = len(gdf_20[gdf_20["CULT_COD"] == cult_cod])
    print(f"Class {cult_cod}:")
    print(f"  Total area: {class_total_area}")
    print(f"  80% area: {class_80_area}")
    print(f"  20% area: {class_20_area}")
    print(f"  Number of polygons in 80% subset: {class_80_polygons}")
    print(f"  Number of polygons in 20% subset: {class_20_polygons}")

Total area: 13818824.478016853
Desired 80% area: 11055059.582413483
Actual 80% subset area: 11093171.60270363
Number of polygons in 80% subset: 747
Number of polygons in 20% subset: 213
Class 311:
  Total area: 11947918.050465232
  80% area: 9580755.1758393
  20% area: 2367162.8746259315
  Number of polygons in 80% subset: 626
  Number of polygons in 20% subset: 180
Class 351:
  Total area: 87226.02352941177
  80% area: 83322.02352941177
  20% area: 3904.0
  Number of polygons in 80% subset: 5
  Number of polygons in 20% subset: 1
Class 321:
  Total area: 1705228.1409787303
  80% area: 1367390.1402914408
  20% area: 337838.0006872894
  Number of polygons in 80% subset: 104
  Number of polygons in 20% subset: 28
Class 36:
  Total area: 56124.35
  80% area: 46204.35
  20% area: 9920.0
  Number of polygons in 80% subset: 9
  Number of polygons in 20% subset: 3
Class 341:
  Total area: 21403.0
  80% area: 14575.0
  20% area: 6828.0
  Number of polygons in 80% subset: 2
  Number of polygons

In [9]:
gdf_80 = gpd.GeoDataFrame(
    gdf_80, crs="EPSG:32631", geometry=gdf_80["geometry"]
)
gdf_20 = gpd.GeoDataFrame(
    gdf_20, crs="EPSG:32631", geometry=gdf_20["geometry"]
)

In [10]:
gdf_80.to_file("images/out/80.shp")

gdf_20.to_file("images/out/20.shp")

import masked images

In [11]:
EnMAP_masked_path = "images/out/EnMAP_glob_masked.tif"

S2_masked_path = "images/out/S2_glob_masked.tif"

Cropping images with plots

EnMAP

In [12]:
output_image_path = "images/out/EnMAP_20.tif"  ## to be changed 80/20perc

# Open the source raster
with rasterio.open(EnMAP_masked_path) as src:
    # Read the GeoDataFrame with the polygons
    geoms = gdf_20.geometry.values  ## to be changed 80/20perc

    # Create a mask
    out_image, out_transform = mask(src, geoms, crop=False, nodata=-10000)

    # Update metadata
    out_meta = src.meta.copy()
    out_meta.update(
        {
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "nodata": -10000,
        }
    )

# Write the masked raster to a new file
with rasterio.open(output_image_path, "w", **out_meta) as dst:
    for i in range(224):
        dst.write(out_image[i], i + 1)  # i + 1 is the band index


src.close()
dst.close()

S2

In [13]:
output_image_path = "images/out/S2_80.tif"  ## to be changed 80/20perc

# Open the source raster
with rasterio.open(S2_masked_path) as src:
    # Read the GeoDataFrame with the polygons
    geoms = gdf_80.geometry.values  ## to be changed 80/20perc

    # Create a mask
    out_image, out_transform = mask(src, geoms, crop=False, nodata=-10000)

    # Update metadata
    out_meta = src.meta.copy()
    out_meta.update(
        {
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "nodata": -10000,
        }
    )

# Write the masked raster to a new file
with rasterio.open(output_image_path, "w", **out_meta) as dst:
    for i in range(10):
        dst.write(out_image[i], i + 1)  # i + 1 is the band index


src.close()
dst.close()

Rasterizing polygons

In [15]:
img_ref = "images/out/EnMAP_80.tif"  ## to be change 20/80
rasterized_80_path = "images/out/rasterized_80.tif"  ## to be change 20/80
no_data = -10000

# Open the source image to get metadata
src = rasterio.open(img_ref, "r")
out_meta = src.meta.copy()
out_meta.update(
    {"count": 2, "nodata": no_data}  # We want 2 bands in the output
)

# Create the output raster with 2 bands
dst = rasterio.open(
    rasterized_80_path, "w+", **out_meta
)  ## to be change 20/80

# Prepare arrays for the two bands
cult_cod_arr = dst.read(1, out_shape=(src.height, src.width))
index_arr = dst.read(1, out_shape=(src.height, src.width))

# This is where we create a generator of geom, value pairs to use in rasterizing
geom_col = gdf_80.geometry  ## to be change 20/80
code_col = gdf_80["CULT_COD"].astype(int)  ## to be change 20/80
index_col = gdf_80.index  ## to be change 20/80

# Create shapes generators
shapes_cult_cod = ((geom, value) for geom, value in zip(geom_col, code_col))
shapes_index = ((geom, index) for geom, index in zip(geom_col, index_col))

# Rasterize CULT_COD band
cult_cod_arr = features.rasterize(
    shapes=shapes_cult_cod,
    fill=no_data,
    out=cult_cod_arr,
    transform=dst.transform,
)

# Rasterize index band
index_arr = features.rasterize(
    shapes=shapes_index, fill=no_data, out=index_arr, transform=dst.transform
)

# Write the two bands to the output file
dst.write_band(1, cult_cod_arr)
dst.write_band(2, index_arr)

print("Rasterization is done")

# Close rasterio objects
src.close()
dst.close()

Rasterization is done
